In [1]:
import json
import numpy as np
import pandas as pd
#import geopandas as gpd
import json
import re
import os
import matplotlib 
from tqdm.notebook import tqdm
from opencage.geocoder import OpenCageGeocode

pd.set_option('display.max_columns', 150)


df = pd.read_csv('../../data/raw/listing_price.csv')

# remove col name from row values:
for col in df.columns:
    df[col] = df[col].str.replace(col,'')
    
# change name of the df
cols = {
    'Sijainti' : 'address',
    'Tyyppi': 'type',
    'Omistusmuoto': 'form_of_ownership',
    'Huoneistoselitelmä':'rooms',
    'Asuintilojen pinta-ala': 'living_area',
    'Kerrokset':'floors',
    'Rakennusvuosi':'construction_year',
    'Velaton hinta': 'debt_free_price',
    'Myyntihinta': 'selling_price',
    'Velkaosuus': 'debt_ratio',
    'Neliöhinta': 'price_m2',
    'Yhtiövastike':'company_compensation',
    'Vesimaksu': 'water_charge',
    'Lisätiedot maksuista': 'additional_charge',
    'Parveke': 'balcony',
    'Hissi': 'elavator',
    'Lämmitysjärjestelmä': 'heating_system',
    'Taloyhtiöön kuuluu': 'building_facilities',
    'Taloyhtiön autopaikat': 'parking_type',
    'Energialuokka':'energy_class',
    'Tontin omistus': 'land_ownership',
    'Kokonaispinta-ala':'total_area',
    'Vapautuminen':'released_year',
    'Asuntoon kuuluu': 'apartment_includes',
    'Asunnon kunto':'condition',
    'Lämmitysjärjestelmän kuvaus':'heating_system_type',
    'Rakennus- ja pintamateriaalit':'building_surface_materials',
    'Keittiön kuvaus':'kitchen_description',
    'Kylpyhuoneen kuvaus': 'bathroom_description',
    'Olohuoneen kuvaus':'living_room_description',
    'Kattotyyppi':'roof_type',
    'Taloyhtiön nimi':'housing_company_name',
    'Tehdyt remontit':'rennovations_made',
    'Tulevat remontit': 'upcoming_renovations',
    'Tontin koko':'building_ground',
}

df = df.rename(columns=cols)

# filter form of ownership == Omistusasunto: owning         
df = df[df.form_of_ownership=='Omistusasunto']

# extract municipality
df['municipality'] = df['address'].map(lambda s: str(s).split()[0])

# extract postcode
df['postcode'] = df['address'].map(lambda s: re.findall('\d{5}',str(s)) )
df['postcode'] = df['postcode'].map(lambda x: x[0] if len(x)>0 else None)

df['property_type'] = df.type.map(lambda s: str(s).split()[0])

# number of rooms
rooms_type = {'3 huonetta': 3,
              'Kaksio': 2,
              '4 huonetta': 4,
              'Yksiö': 1,
              '5 huonetta': 5,
              'Yli 5 huonetta':'5+',
              'Ei tiedossa': None             
}

df['number_of_rooms'] = df['Huoneita'].replace(rooms_type)

df['price'] = df['price'].map(lambda x: re.findall('\d{4,}', x) ) 
df['price'] = df['price'].map(lambda x: x[0] if len(x) >0 else None)

# clean the area colummns: remove m2, replace ',' with ',', and then urn them into numeric values
for col in ['living_area', 'total_area']:
    df[col] = df[col].map(lambda x: str(x).replace(' m²', ''))
    df[col] = df[col].map(lambda x: str(x).replace(',', '.'))
    df[col] = pd.to_numeric(df[col], errors='coerce')

# check if the apartment has private sauna:
sauna = ['Asunnossa on saunaTaloyhtiössä on sauna', 'Asunnossa on sauna']
df['private_sauna'] = np.where(df['Sauna'].isin(sauna), True, False)    

# construction year
df['construction_year'] = df['construction_year'].map(lambda x: re.sub(r'(\d{4})(.*)?',  r"\1" , x) if not pd.isna(x) else x )
df['construction_year'] = df['construction_year'].astype(float)

df['price'] = df['price'].astype(float)
df['price_m2_compute'] = np.round(df['price']) / df['living_area']

# elavator
df['elavator'] = np.where(df['Sauna'].isin(['Taloyhtiössä on hissi']), True, False)

# source of the advertisement
df['source'] = 'https://www.etuovi.com/kohde/' + df['Kohdenumero'] + '?haku=M1618619882'

# translate condition to English
condition = { 'Tyydyttävä' : 'Satisfying',
             'Välttävä': 'Satisfying',
             'Hyvä': 'Good',
             'Ei luokiteltu' : 'Not classified',
             np.nan: 'Not classified',  
}
df.condition = df.condition.replace(condition)

# save file

df.to_csv('../../data/processed/listing_price.csv', sep= ';', index= False)